In [1]:
import List_to_Pytorch_Dataset
import torch
from torch import nn
from torch.utils.data import DataLoader
import CNN1D
import Smaller_input_CNN
import DeepCNN
from statistics import mean

In [2]:
print(torch.__version__)

1.12.1+cpu


In [3]:
dataset = List_to_Pytorch_Dataset.MobifallData(augment=False)

STD 180
WAL 183
JOG 183
JUM 183
STU 200
STN 200
SCH 200
SIT 190
CHU 114
CSI 200
CSO 197
FOL 192
FKL 192
BSC 191
SDL 192


In [4]:
print(f"Dataset size: {len(dataset)}")
print(f"Data sample size: {len(dataset[0][0][0])}")

Dataset size: 2797
Data sample size: 250


In [5]:
device = 'cpu' #cuda:0' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')
torch.cuda.empty_cache()

Using cpu device


In [6]:
labels = { 0  : "STD",
           1  : "WAL",
           2  : "JOG",
           3  : "JUM",
           4  : "STU",
           5  : "STN",
           6  : "SCH",
           7  : "SIT",
           8  : "CHU",
           9  : "CSI",
           10 : "CSO",
           11 : "FOL",
           12 : "FKL",
           13 : "BSC",
           14 : "SDL"
           #15 : "PFF"
           }

classification_map = {}
classification_map["Falls"] = {"TP" : 0,
                                 "FP" : 0,
                                 "TN" : 0,
                                 "FN" : 0}
for key, value in labels.items():
    classification_map[value] = {"TP" : 0,
                                 "FP" : 0,
                                 "TN" : 0,
                                 "FN" : 0}

In [7]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X = X.to(device)
        y = y.to(device)
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch%10 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


In [8]:
def is_fall(label):
    return label in [11, 12, 13, 14]

In [9]:
def valid_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    valid_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X = X.to(device)
            y = y.to(device)
            
            pred = model(X)

            for key, value in labels.items():
                for i in range(len(y)):
                    if y[i] == key:
                        if pred[i].argmax() == key:
                            classification_map[value]["TP"] += 1
                        else:
                            classification_map[value]["FN"] += 1
                    elif pred[i].argmax() == key:
                        classification_map[value]["FP"] += 1
                    else:
                        classification_map[value]["TN"] += 1
                        
            #this is for fall/not-fall classification
            for i in range(len(y)):
                if is_fall(y[i]):
                    if is_fall(pred[i].argmax()):
                        classification_map["Falls"]["TP"] += 1
                    else:
                        classification_map["Falls"]["FN"] += 1
                elif is_fall(pred[i].argmax()):
                    classification_map["Falls"]["FP"] += 1
                else:
                    classification_map["Falls"]["TN"] += 1
                    

            #model loss and accuracy
            valid_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    #model stats
    valid_loss /= num_batches
    correct /= size
    print(f"Validation Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {valid_loss:>8f} \n\n")
    return 100*correct

In [12]:
def main():
    model = DeepCNN.CNN().to(device)
    model.train()

    #this is for mobile use adaptation
    backend = "qnnpack"
    model.qconfig = torch.quantization.get_default_qat_qconfig(backend)
    model = torch.quantization.prepare_qat(model, inplace=False)

    learning_rate = 0.001
    epochs = 40

    # Initialize the loss function and optimizer
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    #creating datasets
    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size
    train_set, test_set = torch.utils.data.random_split(dataset, [train_size, test_size])
    

    #Dataloaders for the datasets
    train_dataloader = DataLoader(train_set, batch_size = 64, shuffle = True)
    test_dataloader = DataLoader(test_set, batch_size = 64, shuffle = True)
    
    model.train()
    for i in range(epochs):
        print(i)
        train_loop(train_dataloader, model, loss_fn, optimizer)
    
    model.eval()
    model.to('cpu')

    model = torch.quantization.convert(model.eval(), inplace=False)
    model_accuracy = valid_loop(test_dataloader, model, loss_fn)

    return model, model_accuracy


In [16]:
model_accuracy = 0
while model_accuracy < 90:
    metric_list = []
    model_accuracies = []
    #for _ in range(5):
    result = main()
    model = result[0]
    model_accuracy = result[1]
    if model_accuracy < 90:
        continue
    model_accuracies.append(model_accuracy)
    stats_map = {}
    for key, value in classification_map.items():
        stats_map[key] = {
            "Specificity" : float(classification_map[key]["TN"]) / float(classification_map[key]["TN"] + classification_map[key]["FP"]),
            "Recall" : float(classification_map[key]["TP"]) / float(classification_map[key]["TP"] + classification_map[key]["FN"]),
            "Precision" : float(classification_map[key]["TP"]) / float(classification_map[key]["TP"] + classification_map[key]["FP"]),
            "Accuracy" : float(classification_map[key]["TP"] + classification_map[key]["TN"]) / float(classification_map[key]["TP"] + classification_map[key]["TN"] + classification_map[key]["FP"] + classification_map[key]["FN"])
        }
        stats_map[key]["F-score"] = 2.0 / float((1.0 / float(stats_map[key]["Precision"])) + (1.0 / float(stats_map[key]["Recall"])))
    metric_list.append(stats_map)
    for key, value in classification_map.items():
        classification_map[key] = {"TP" : 0,
                                "FP" : 0,
                                "TN" : 0,
                                "FN" : 0}


0
loss: 2.702601  [    0/ 2237]
loss: 2.712802  [  640/ 2237]
loss: 2.703469  [ 1280/ 2237]
loss: 2.703236  [ 1920/ 2237]
1
loss: 2.698845  [    0/ 2237]
loss: 2.683990  [  640/ 2237]
loss: 2.697668  [ 1280/ 2237]
loss: 2.725873  [ 1920/ 2237]
2
loss: 2.664810  [    0/ 2237]
loss: 2.717226  [  640/ 2237]
loss: 2.846455  [ 1280/ 2237]
loss: 2.924891  [ 1920/ 2237]
3
loss: 3.014465  [    0/ 2237]
loss: 2.986933  [  640/ 2237]
loss: 4.003813  [ 1280/ 2237]
loss: 3.479744  [ 1920/ 2237]
4
loss: 4.332331  [    0/ 2237]
loss: 6.253130  [  640/ 2237]
loss: 8.683530  [ 1280/ 2237]
loss: 9.185909  [ 1920/ 2237]
5
loss: 15.485758  [    0/ 2237]
loss: 10.666433  [  640/ 2237]
loss: 27.168232  [ 1280/ 2237]
loss: 40.453499  [ 1920/ 2237]
6
loss: 57.084961  [    0/ 2237]
loss: 104.667557  [  640/ 2237]
loss: 132.851089  [ 1280/ 2237]
loss: 293.307526  [ 1920/ 2237]
7
loss: 251.332382  [    0/ 2237]
loss: 707.068359  [  640/ 2237]
loss: 878.795959  [ 1280/ 2237]
loss: 1230.186768  [ 1920/ 2237]
8
lo

KeyboardInterrupt: 

In [ ]:
for key, value in metric_list[0].items():
    if key == "FOL" or key =="FKL" or key == "BSC" or key =="SDL" or key =="Falls":
        specificity = mean([elem[key]["Specificity"] for elem in metric_list])
        recall = mean([elem[key]["Recall"] for elem in metric_list])
        precision = mean([elem[key]["Precision"] for elem in metric_list])
        fScore = mean([elem[key]["F-score"] for elem in metric_list])
        print("Specificity of " + key + ": " + str(specificity))
        print("Recall of " + key + ": " + str(recall))
        print("Precision of " + key + ": " + str(precision))
        print("F1-score of " + key + ": " + str(fScore))
        print("")
print("Model accuracy: " + str(mean(model_accuracies)))

Specificity of Falls: 0.99226899006013
Recall of Falls: 0.37921992789249426
Precision of Falls: 0.9483606557377049
F1-score of Falls: 0.5417934909857176

Specificity of FOL: 0.9810162991371045
Recall of FOL: 0.2948051948051948
Precision of FOL: 0.5341176470588235
F1-score of FOL: 0.3799163179916318

Specificity of FKL: 0.9860415864458991
Recall of FKL: 0.21921182266009853
Precision of FKL: 0.5510835913312694
F1-score of FKL: 0.31365638766519827

Specificity of BSC: 0.99713384924047
Recall of BSC: 0.2755798090040928
Precision of BSC: 0.8706896551724138
F1-score of BSC: 0.4186528497409327

Specificity of SDL: 0.9928325688073395
Recall of SDL: 0.22418478260869565
Precision of SDL: 0.6875
F1-score of SDL: 0.3381147540983606

Model accuracy: 92.14285714285714


In [14]:
from torch.utils.mobile_optimizer import optimize_for_mobile
def save_model(model):
    torchscript_model = torch.jit.script(model)
    torchscript_model_optimized = optimize_for_mobile(torchscript_model)
    torch.jit.save(torchscript_model_optimized, "MIS_RFD_model_quantized.pt")
    #torchscript_model_optimized._save_for_lite_interpreter("MIS_RFD_model_quantized.ptl")

In [15]:
save_model(model)